In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from scipy import misc
import cv2
import numpy as np
from library import facenet
from library import detect_face
import os
import time
import pickle
import PIL
from PIL import Image
from datetime import datetime

input_video="example_mov.avi"
modeldir = './model/20191012-185253.pb'
classifier_filename = './class/classifier.pkl'
npy='./npy'
train_img="./face_dataset"
names = []

# def markAttendance(name):
#     with open('Y:\\Masters_Content\\Software_development_for_AI\\Project 2\\Attendance Tracking 1\\Face-Detection-and-Recogition-TF-FN-OCV-master\\Attendance.csv','r+') as f:
#         myDataList = [x.strip() for x in f.readlines()]
#         nameList = []
#         for line in myDataList:
#             entry = line.split(',')
#             nameList.append(entry[0])
#         if name not in nameList:
#             now = datetime.now()
#             dtString = now.strftime('%H:%M:%S')
#             f.writelines(f'{name},{dtString}\n') 

with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = detect_face.create_mtcnn(sess, npy)

        minsize = 20  # minimum size of face
        threshold = [0.6, 0.7, 0.7]  # three steps's threshold
        factor = 0.709  # scale factor
        margin = 44
        frame_interval = 3
        batch_size = 1000
        image_size = 182
        input_image_size = 160
        
        HumanNames = os.listdir(train_img)
        HumanNames.sort()

        print('Loading Modal')
        facenet.load_model(modeldir)
        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        embedding_size = embeddings.get_shape()[1]


        classifier_filename_exp = os.path.expanduser(classifier_filename)
        with open(classifier_filename_exp, 'rb') as infile:
            (model, class_names) = pickle.load(infile)

        video_capture = cv2.VideoCapture(0)
        c = 0


        print('Start Recognition')
        prevTime = 0
        while True:
            ret, frame = video_capture.read()

            frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)    #resize frame (optional)

            curTime = time.time()+1    # calc fps
            timeF = frame_interval

            if (c % timeF == 0):
                find_results = []

                if frame.ndim == 2:
                    frame = facenet.to_rgb(frame)
                frame = frame[:, :, 0:3]
                bounding_boxes, _ = detect_face.detect_face(frame, minsize, pnet, rnet, onet, threshold, factor)
                nrof_faces = bounding_boxes.shape[0]
#                 print('Detected_FaceNum: %d' % nrof_faces)

                if nrof_faces > 0:
                    det = bounding_boxes[:, 0:4]
                    img_size = np.asarray(frame.shape)[0:2]

                    cropped = []
                    scaled = []
                    scaled_reshape = []
                    bb = np.zeros((nrof_faces,4), dtype=np.int32)

                    for i in range(nrof_faces):
                        emb_array = np.zeros((1, embedding_size))

                        bb[i][0] = det[i][0]
                        bb[i][1] = det[i][1]
                        bb[i][2] = det[i][2]
                        bb[i][3] = det[i][3]

                        # inner exception
                        if bb[i][0] <= 0 or bb[i][1] <= 0 or bb[i][2] >= len(frame[0]) or bb[i][3] >= len(frame):
                            print('Face is very close!')
                            continue

                        cropped.append(frame[bb[i][1]:bb[i][3], bb[i][0]:bb[i][2], :])
                        cropped[i] = facenet.flip(cropped[i], False)
                        scaled.append(np.array(Image.fromarray(cropped[i]).resize((image_size, image_size),resample=PIL.Image.BILINEAR)))
                        scaled[i] = cv2.resize(scaled[i], (input_image_size,input_image_size),
                                               interpolation=cv2.INTER_CUBIC)
                        scaled[i] = facenet.prewhiten(scaled[i])
                        scaled_reshape.append(scaled[i].reshape(-1,input_image_size,input_image_size,3))
                        feed_dict = {images_placeholder: scaled_reshape[i], phase_train_placeholder: False}
                        emb_array[0, :] = sess.run(embeddings, feed_dict=feed_dict)
                        predictions = model.predict_proba(emb_array)
#                         print('predictions:',predictions)
#                         pred_ = sorted(predictions[0])
#                         print('predict:',pred_)
                        best_class_indices = np.argmax(predictions, axis=1)
#                         print('best_class_indices:',best_class_indices)
                        best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]
                        #print("predictions")
                        print('Class',best_class_indices,'is detected with accuracy ',best_class_probabilities)

                        # print(best_class_probabilities)
                        if best_class_probabilities>0.53:
                            cv2.rectangle(frame, (bb[i][0], bb[i][1]), (bb[i][2], bb[i][3]), (0, 255, 0), 2)    #boxing face

                            #plot result idx under box
                            text_x = bb[i][0]
                            text_y = bb[i][3] + 20
#                             print('Result Indices: ', best_class_indices)
#                             print(HumanNames)
                            for H_i in HumanNames:
                                if HumanNames[best_class_indices[0]] == H_i:
                                    result_names = HumanNames[best_class_indices[0]].upper()
                                    names.append(result_names)
#                                     markAttendance(result_names)
                                    cv2.putText(frame, result_names, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                                1, (0, 0, 255), thickness=1, lineType=2)
                        else:
                            continue
            # c+=1
            cv2.imshow('Video', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        video_capture.release()
        cv2.destroyAllWindows()
        names = list( dict.fromkeys(names))

Loading Modal
Model filename: ./model/20191012-185253.pb
Start Recognition
Class [0] is detected with accuracy  [0.56132717]
Class [0] is detected with accuracy  [0.64673598]
Class [0] is detected with accuracy  [0.59138048]
Class [0] is detected with accuracy  [0.52586484]
Class [3] is detected with accuracy  [0.47989467]
Class [0] is detected with accuracy  [0.54037404]
Class [0] is detected with accuracy  [0.55857187]
Class [0] is detected with accuracy  [0.50373569]
Class [0] is detected with accuracy  [0.5647068]
Class [0] is detected with accuracy  [0.59338285]
Class [0] is detected with accuracy  [0.61436563]
Class [3] is detected with accuracy  [0.48286991]
Class [0] is detected with accuracy  [0.57978845]
Class [0] is detected with accuracy  [0.59972757]
Class [0] is detected with accuracy  [0.58717591]
Class [0] is detected with accuracy  [0.64318968]
Class [0] is detected with accuracy  [0.60421859]
Class [0] is detected with accuracy  [0.48466395]
Class [0] is detected with

Class [0] is detected with accuracy  [0.57360058]
Class [0] is detected with accuracy  [0.6960236]
Class [0] is detected with accuracy  [0.56743658]
Class [0] is detected with accuracy  [0.62711063]
Class [3] is detected with accuracy  [0.47689572]
Class [0] is detected with accuracy  [0.76621952]
Class [0] is detected with accuracy  [0.68431875]
Class [0] is detected with accuracy  [0.77349407]
Class [0] is detected with accuracy  [0.66891623]
Class [0] is detected with accuracy  [0.75613376]
Class [0] is detected with accuracy  [0.76554349]
Class [0] is detected with accuracy  [0.7355337]
Class [0] is detected with accuracy  [0.71364709]
Class [0] is detected with accuracy  [0.73565063]
Class [0] is detected with accuracy  [0.56843544]
Class [0] is detected with accuracy  [0.6757427]
Class [0] is detected with accuracy  [0.75322529]
Class [0] is detected with accuracy  [0.76559866]
Class [0] is detected with accuracy  [0.72984709]
Class [0] is detected with accuracy  [0.62968276]
Cla

Class [0] is detected with accuracy  [0.68745118]
Class [0] is detected with accuracy  [0.61951775]
Class [0] is detected with accuracy  [0.53620736]
Class [0] is detected with accuracy  [0.60264372]
Class [0] is detected with accuracy  [0.58539711]
Class [0] is detected with accuracy  [0.5673802]
Class [0] is detected with accuracy  [0.61245857]
Class [0] is detected with accuracy  [0.76056678]
Class [0] is detected with accuracy  [0.75815921]
Class [0] is detected with accuracy  [0.64909472]
Class [0] is detected with accuracy  [0.55334012]
Class [3] is detected with accuracy  [0.88513507]
Class [0] is detected with accuracy  [0.8087438]
Class [3] is detected with accuracy  [0.94290579]
Class [0] is detected with accuracy  [0.71748161]
Class [0] is detected with accuracy  [0.74792776]
Class [0] is detected with accuracy  [0.76904043]
Class [0] is detected with accuracy  [0.78603472]
Class [0] is detected with accuracy  [0.73846581]
Class [0] is detected with accuracy  [0.72095881]
Cl